In [20]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense, Dropout,Input
from keras.optimizers import Adam
import numpy as np
import joblib

In [21]:
# Langkah 1: Membaca data dari CSV
data = pd.read_csv('data_set.csv')

# Menampilkan data pertama untuk memastikan formatnya
print(f"Jumlah data: {data.shape[0]} baris dan {data.shape[1]} kolom")
print(data.head())

# Memastikan tidak ada nilai NaN atau kosong di dalam dataset
print(f"Data yang memiliki nilai NaN: \n{data.isnull().sum()}")


Jumlah data: 150 baris dan 4 kolom
    Level_Emosi Tipe_Emosi Sumber_Emosi  \
0  Sangat Buruk     Kecewa      Sekolah   
1  Sangat Buruk  Frustrasi      Sekolah   
2  Sangat Buruk    Bingung      Sekolah   
3  Sangat Buruk    Bahagia      Sekolah   
4  Sangat Buruk      Cemas      Sekolah   

                                 Pertanyaan_Analisis  
0  Apa yang membuat kamu merasa kecewa di Sekolah...  
1  Apa yang membuat kamu merasa frustrasi di Seko...  
2  Apa yang membuat kamu merasa bingung di Sekola...  
3  Apa yang membuat kamu merasa bahagia di Sekola...  
4  Apa yang membuat kamu merasa cemas di Sekolah?...  
Data yang memiliki nilai NaN: 
Level_Emosi            0
Tipe_Emosi             0
Sumber_Emosi           0
Pertanyaan_Analisis    0
dtype: int64


In [22]:
# Langkah 2: Menyiapkan data untuk model
# Menggabungkan Level_Emosi, Tipe_Emosi, dan Sumber_Emosi menjadi satu kolom input
data['Input_Text'] = data['Level_Emosi'] + " " + data['Tipe_Emosi'] + " " + data['Sumber_Emosi']

# Menggunakan TfidfVectorizer untuk mengubah teks menjadi fitur numerik
vectorizer = TfidfVectorizer(max_features=1500)  # Menggunakan 1500 fitur teratas untuk menghindari overfitting
X = vectorizer.fit_transform(data['Input_Text']).toarray()  # Fitur numerik

In [23]:
# Langkah 3: Encoding target (Pertanyaan_Analisis) menjadi label numerik
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['Pertanyaan_Analisis'])  # Target yang ingin diprediksi (pertanyaan analisis)


In [24]:
# Langkah 4: Memeriksa jumlah data dan membagi dataset
# Membagi data menjadi training dan testing set (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [25]:
# Langkah 5: Membangun model Neural Network (ANN)
model = Sequential()

# Layer input (dengan 1500 fitur dari TfidfVectorizer)
model.add(Input(shape=(X_train.shape[1],)))  # Menggunakan Input layer untuk menentukan bentuk input
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))  # Dropout untuk mengurangi overfitting
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))  # Dropout untuk mengurangi overfitting
model.add(Dense(64, activation='relu'))

# Output layer (jumlah pertanyaan analisis)
model.add(Dense(len(set(y)), activation='softmax'))  # Jumlah pertanyaan yang berbeda

# Kompilasi model
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [26]:
# Langkah 6: Melatih model
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))  # Mengurangi epoch agar sesuai dengan data kecil

Epoch 1/50


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.0163 - loss: 3.4062 - val_accuracy: 0.0333 - val_loss: 3.3649
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1016 - loss: 3.3454 - val_accuracy: 0.1333 - val_loss: 3.3279
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1416 - loss: 3.3019 - val_accuracy: 0.2667 - val_loss: 3.2774
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2327 - loss: 3.2388 - val_accuracy: 0.2000 - val_loss: 3.2071
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2635 - loss: 3.1579 - val_accuracy: 0.2333 - val_loss: 3.1170
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2448 - loss: 3.0428 - val_accuracy: 0.2667 - val_loss: 3.0090
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3605 - loss: 2.8628 - val_accuracy: 0.3000 - val_loss: 2.8713
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4041 - loss: 2.6660 - val_accuracy: 0.4333 - val_loss: 2.6860
Epoch 9/50
8/8 ━━━

In [27]:
# Langkah 7: Evaluasi model
score = model.evaluate(X_test, y_test)
print(f"Model Accuracy: {score[1]*100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 1.0000 - loss: 0.0022
Model Accuracy: 100.00%


In [28]:
# Menyimpan model untuk penggunaan selanjutnya
model.save("../app/models/emotion_tracking_model.keras")

# Menyimpan TfidfVectorizer dan LabelEncoder untuk penggunaan selanjutnya
joblib.dump(vectorizer, '../app/models/vectorizer.pkl')  # Menyimpan TfidfVectorizer
joblib.dump(label_encoder, '../app/models/label_encoder.pkl')  # Menyimpan LabelEncoder

print("Model, TfidfVectorizer, dan LabelEncoder berhasil disimpan!")


Model, TfidfVectorizer, dan LabelEncoder berhasil disimpan!
